## Explore some real pulsar data

This notebook looks at some MSPs from the ANTF pulsar catalogue


In [1]:
from psrqpy import QueryATNF
query = QueryATNF(params=['Name','PSRJ', 'F0','F1','RAJD','DecJD','Dist'])
df = query.pandas

ModuleNotFoundError: No module named 'psrqpy'

## Case 1: Two random arrays:

In [10]:
import numpy as np

xdims = 6
ydims = 5


Pxy = np.ones((xdims,ydims)) * np.random.uniform(0,1,size=((xdims,ydims)))
Pyy = np.ones((ydims,ydims)) * np.random.uniform(0,1,size=((ydims,ydims)))


print(Pxy)
print('-------------')
print(Pyy)

[[0.00649553 0.2280556  0.25289036 0.75818436 0.99433209]
 [0.76668165 0.27969992 0.08450494 0.52369974 0.85647668]
 [0.39603961 0.87040198 0.7719638  0.90973154 0.80300699]
 [0.28714274 0.25692951 0.46169375 0.70232583 0.99485387]
 [0.6329199  0.62444154 0.53151288 0.7146627  0.5333774 ]
 [0.41945753 0.283859   0.16528925 0.76530933 0.92639165]]
-------------
[[0.72305836 0.16218958 0.69167692 0.34840618 0.14980758]
 [0.0971603  0.86304592 0.93636447 0.47552254 0.34577871]
 [0.86576197 0.44704838 0.20606991 0.32406929 0.93127207]
 [0.43402216 0.51324759 0.72790838 0.47066741 0.55565692]
 [0.22050525 0.64381344 0.79126592 0.1489973  0.85857497]]


In [11]:
#Definition 1
K1 = Pxy @ np.linalg.inv(Pyy)

#Definition 2
Q,R = np.linalg.qr(Pyy)
Qb = Pxy @ Q.T
K2 = np.linalg.solve(R,Qb.T).T


#Subtract
print("Difference in Kalman solutions:")
print(K2 - K1)

[[-0.20572673 -0.3155973   1.24409317 -3.78211036  3.25814099]
 [-0.30850323  0.85256564 -1.38817742 -0.4827416   1.76057107]
 [-1.50111874 -0.02411269 -1.08522296 -0.0948091   2.57458821]
 [-0.62289272  0.47798996  0.41272758 -2.92731728  2.65693471]
 [-1.42731017  0.09328294 -1.75265245  1.29822219  1.9927024 ]
 [-0.44119323 -0.09334903 -0.11863634 -1.90800388  3.06881847]]


Hmm! Looks like these two definitions are not equivalent generally? What about if we try a simpler matrix:


## Case 2: Random, diagonal matrices

In [21]:
Pxy = np.eye(N=xdims,M=ydims) * np.random.uniform(0,1,size=((xdims,ydims)))
Pyy = np.eye(N=ydims,M=ydims) * np.random.uniform(0,1,size=((ydims,ydims)))

print(Pxy)
print('-------------')
print(Pyy)



#Definition 1
K1 = Pxy @ np.linalg.inv(Pyy)

#Definition 2
Q,R = np.linalg.qr(Pyy)
Qb = Pxy @ Q.T
K2 = np.linalg.solve(R,Qb.T).T


#Subtract
print("Difference in Kalman solutions:")
print(K2 - K1)

[[0.97359649 0.         0.         0.         0.        ]
 [0.         0.94309432 0.         0.         0.        ]
 [0.         0.         0.34717288 0.         0.        ]
 [0.         0.         0.         0.65022376 0.        ]
 [0.         0.         0.         0.         0.36193577]
 [0.         0.         0.         0.         0.        ]]
-------------
[[0.3420506  0.         0.         0.         0.        ]
 [0.         0.14578382 0.         0.         0.        ]
 [0.         0.         0.71017011 0.         0.        ]
 [0.         0.         0.         0.47727093 0.        ]
 [0.         0.         0.         0.         0.27206902]]
Difference in Kalman solutions:
[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 -0.0000000e+00
   0.0000000e+00]
 [ 0.0000000e+00 -8.8817842e-16  0.0000000e+00 -0.0000000e+00
   0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 -0.0000000e+00
   0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000

Ok this now works! So is this something to do with the off-diagonal terms?


## Case 3: Off diagonal terms

In [22]:
Pxy = np.eye(N=xdims,M=ydims) * np.random.uniform(0,1,size=((xdims,ydims)))
Pyy = np.eye(N=ydims,M=ydims) * np.random.uniform(0,1,size=((ydims,ydims)))


Pxy[2,3] = np.random.uniform(0,1)
Pyy[4,2] = np.random.uniform(0,1)


print(Pxy)
print('-------------')
print(Pyy)



#Definition 1
K1 = Pxy @ np.linalg.inv(Pyy)

#Definition 2
Q,R = np.linalg.qr(Pyy)
Qb = Pxy @ Q.T
K2 = np.linalg.solve(R,Qb.T).T


#Subtract
print("Difference in Kalman solutions:")
print(K2 - K1)

[[0.54625028 0.         0.         0.         0.        ]
 [0.         0.49088819 0.         0.         0.        ]
 [0.         0.         0.44880401 0.1341587  0.        ]
 [0.         0.         0.         0.58420655 0.        ]
 [0.         0.         0.         0.         0.17569771]
 [0.         0.         0.         0.         0.        ]]
-------------
[[0.26442079 0.         0.         0.         0.        ]
 [0.         0.45324211 0.         0.         0.        ]
 [0.         0.         0.54593748 0.         0.        ]
 [0.         0.         0.         0.71285484 0.        ]
 [0.         0.         0.51887914 0.         0.03556407]]
Difference in Kalman solutions:
[[ 0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  2.22044605e-16 -0.00000000e+00 -0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  2.22044605e-16  0.00000000e+00
  -1.19941261e+01]
 [ 0.00000000e+00  0.00000000e+00 -0.00000000e+00 -1.110223

aaaaaand now its broken again!



## Case 4, the solution


The alternative definition of the Kalman gain does work, we just need to be mindful about what needs transposing